In [23]:
import os
from os.path import join
from os import listdir
import pandas as pd
import numpy as np
import tqdm
import pandas as pd
import rouskinhf

## Plot showing the quality of our datasets

Compare replicates:
- R2 score between normalized DMS signals (first row of subplot)
- F1 score between replicate + RNAstructure (second row)

Then bootstrap from the DMS by adding noise proportional to the confidence interval, then predicting the structure with the noisy DMS. Compute the F1 score between the structures (third row of the subplot)

Use pri_miRNA and human_mRNA (called UTR previously). Combine them in one plot, or separate them in two columns if realy different

**Assigned to**: Yves

Use Ploty, and a white background

## Build dataset (no need to re-run)

In [24]:
# # load refs from HF
# refs = list(rouskinhf.get_dataset('human_mRNA').keys()) + list(rouskinhf.get_dataset('pri_miRNA').keys())

# # load reps data from local
# pri = pd.read_feather('pri_miRNA_normalized.feather').drop(columns=['index'])
# pri['dataset'] = 'Pri-miRNA'
# mrna = pd.read_feather('mRNA_normalized.feather').drop(columns=['index'])
# mrna['dataset'] = 'Human mRNA'
# df = pd.concat([pri, mrna])

# # keep only reps that are in HF
# df = df[df['replicate'] != 'Untreated']
# df = df[df['reference'].isin(refs)].reset_index(drop=True)

# # count the number of reference per sample
# df['number_of_replicates'] = df.groupby(['reference', 'dataset', 'plate'])['reference'].transform('count')

# # Count the amount of available reps per dataset
# ref_per_sample_per_dataset = {}
# for dataset in df['dataset'].unique():
#     loc_df = df[df['dataset']==dataset]
#     ref_per_sample_per_dataset[dataset] = {k[0]: v for k, v in dict(loc_df.value_counts(['number_of_replicates']).sort_index()).items()} 

# # drop the reference with less than 2 sample 
# l = len(df)
# df = df[df['number_of_replicates'] == 2]
# print("drop {}/{} references".format(l - len(df), l))


In [25]:

# # Run RNAstructure
# rnastructure_path = '/Users/yvesmartin/lib/RNAstructure/exe'

# def file_name(stage): 
#     return f'{stage}/{file_name.ref}_{file_name.replicate}.{stage}'

# for stage in ['fasta', 'shape', 'ct', 'dot']:
#     os.makedirs(stage, exist_ok=True)
    
# def predict_structure(seq, ref, rep, dms=None):
#     file_name.ref = ref
#     file_name.replicate = rep
#     os.system(f'echo ">{ref}\n{seq}" >{file_name("fasta")}')
#     if dms is None:
#         dms = np.zeros(len(seq))# * -1000.
#     dms = pd.DataFrame({'dms': [float(d) for d in dms], 'seq': list(seq)})
#     dms.index = dms.index + 1
#     dms = dms[dms['seq'].isin(['A', 'C'])]['dms']
#     dms.to_csv(file_name('shape'), sep='\t', header=False)
#     os.system(f"{rnastructure_path}/fold {file_name('fasta')} {file_name('ct')} --DMS {file_name('shape')}")
#     os.system(f"{rnastructure_path}/ct2dot {file_name('ct')}  -1 {file_name('dot')}" )
#     with open(file_name('dot')) as f:
#         lines = f.readlines()
#     energy = float(lines[0].split()[2])
#     structure = lines[2].strip()
#     return energy, structure


# for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
#     seq, ref, rep, dms = row[['sequence', 'reference', 'replicate', 'sub_rate']]
#     energy, structure = predict_structure(seq, ref, rep, dms)
#     df.loc[i, 'energy'] = energy
#     df.loc[i, 'structure'] = structure
    
# # read RNAstructure output
# def read_dot(dot_file):
#     with open(dot_file, 'r') as f:
#         lines = f.readlines()
#     energy, structure = [], []

#     def get_energy(line):
#         return float(line.split(' ')[2].strip())

#     energy.append(get_energy(lines[0]))
#     structure.append(lines[2].strip())
    
#     for line in lines[3:]:
#         if line.startswith('>ENERGY'):
#             energy.append(get_energy(line))
#         else:
#             structure.append(line.strip())
    
#     assert len(energy) == len(structure)
#     return energy, structure

# rna_pred = []
# for file in os.listdir('dot'):
#     energies, structures = read_dot(f'dot/{file}')
#     for idx, (s, e) in enumerate(zip(structures, energies)):
#         rna_pred.append({
#             'reference': file[:-6],
#             'structure': s,
#             'energies': e,
#             'replicate': file.split('_')[-1].split('.')[0],
#             'rank': idx
#         })
    
# rna_pred = pd.DataFrame(rna_pred)

# # merge with df
# df = pd.merge(df, rna_pred, on=['reference', 'replicate'], how='inner')

# df.to_feather('data_quality.feather')

## Plot replicates distribution per dataset

In [26]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# ref_per_sample_per_dataset = pd.DataFrame(ref_per_sample_per_dataset)
# ref_per_sample_per_dataset.index = ['No replicate', '1 replicate']
# fig = make_subplots(rows=1, cols=2, subplot_titles=ref_per_sample_per_dataset.columns, specs=[[{'type':'domain'}, {'type':'domain'}]])

# for i, col in enumerate(ref_per_sample_per_dataset.columns):
#     fig.add_trace(
#         go.Pie(
#             labels=ref_per_sample_per_dataset.index, 
#             values=ref_per_sample_per_dataset[col],
#             name=col,
#             textinfo='label+percent',
#             textposition='auto',
#             rotation=270,
#     ), row=1, col=i+1)

# fig.update_layout(
#     title_text="Number of replicates per reference in each dataset",
#     width=800,
# )
# fig.show()

# # download figure as a pdf
# import plotly.io as pio
# pio.write_image(fig, 'replicates_per_dataset.pdf')

# Load data

In [27]:
import pandas as pd
import numpy as np
import tqdm

df_all = pd.read_feather('data_quality.feather')
df = df_all.sort_values('energy').groupby(['reference', 'replicate']).first().reset_index()


## Compute scores

In [28]:
import plotly.graph_objects as go

# set default font to helvetica light
import plotly.io as pio
pio.templates.default = "plotly_white"
pio.templates["plotly_white"].layout.font.family = "Helvetica Neue, Helvetica, Arial, sans-serif"


def dot2bool(dot):
    return np.array([1 if c == '.' else 0 for c in dot])

def compute_score_between_replicates(df, score):
    scores = []
    for (ref, plate), group in tqdm.tqdm(df.groupby(['reference', 'plate']), total=len(df['reference'].unique())):
        A = group[group['replicate'] == 'A']
        B = group[group['replicate'] == 'B']
        dataset = group['dataset'].values[0]
        scores.append({
            'reference': ref,
            'plate': plate,
            'replicate': 'A',
            score.name: score(A, B),
            'dataset': dataset
        })
        scores.append({
            'reference': ref,
            'plate': plate,
            'replicate': 'B',
            score.name: score(B, A),
            'dataset': dataset
        })
    
    return pd.DataFrame(scores)

def plot_score_histogram(df, scores, score, min_X):
    fig = go.Figure()
    for name, dataset in scores.groupby('dataset'):
        fig.add_trace(go.Histogram(x=dataset[score.name], name=name, 
                            xbins=dict(start=min_X, end=1, size=0.05),
                            group = name,
                            ))

    fig.update_layout(
        title='{} score (N={}, # of {} scores < {} = {}) for df: {}'.format(score.name, len(scores), score.name, min_X, len(scores[scores[score.name] < min_X]), df.custom_name),
        xaxis_title=score.name,
        yaxis_title='count',
        bargap=0.2,
        bargroupgap=0.1,
        xaxis_range=[min_X, 1],
    )
    return fig


import plotly
colors = plotly.colors.qualitative.Plotly

def violin_plot(df, scores, score, min_X):  
    fig = go.Figure()
    for name, dataset in scores.groupby('dataset'):
        fig.add_trace(go.Violin(x=dataset[score.name], name=name + ' (N={:,})'.format(len(dataset)),
                            meanline_visible=True,
                            # dont show outliers
                            marker=dict(color=colors[1 if name == 'Human mRNA' else 2]),
                            points=False
                            ))

    fig.update_layout(
        title='{} score distribution between replicates'.format(score.name),
        xaxis_title=score.name + ' score',
        # yaxis_title='dataset',
        bargap=0.2,
        bargroupgap=0.1,
        xaxis_range=[min_X, 1],
        showlegend=False,
        width=800,    
        paper_bgcolor='white',  # Background color of the entire plot
        plot_bgcolor='white',  # Background color of the plot area
        # add a frame
        margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins
        xaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='lightgrey',
            mirror=True,
            showgrid=False,
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            showline=True,
            linewidth=2,
            linecolor='lightgrey',
            mirror=True,
            showgrid=False,
            gridcolor='white',
            gridwidth=2,
        ),
        font=dict(
        size=18,
    )
    )
    
    return fig
    
def normalize(x, mask=None):
    x = np.array(x)
    mask = x != -1000. if mask is None else mask
    per90 = np.percentile(x, 90)
    y = np.clip(x[mask] / per90, 0, 1)
    x[mask] = y
    return x


### AUROC score


In [29]:
from sklearn.metrics import roc_auc_score

def auroc(row):
    sig = row['sub_rate']
    mask = sig != -1000.
    sig = sig[mask]
    structure = dot2bool(row['structure'])[mask]
    return roc_auc_score(structure, sig)

df_all['auroc'] = df_all.apply(lambda x: auroc(x), axis=1)
df_best = df_all.sort_values('auroc', ascending=False).groupby(['reference', 'plate','replicate']).first().reset_index()

df.custom_name = 'lowest predicted energy'
df_best.custom_name = 'best auroc'

### F1 score

In [30]:
# import f1_score
from sklearn.metrics import f1_score as f1_score_sklearn

import rnastructure
rna = rnastructure.RNAstructure()

def f1_score(x0, x1):
    x0, x1 = x0['structure'].values[0], x1['structure'].values[0]
    return f1_score_sklearn(dot2bool(x0), dot2bool(x1))

f1_score.name = 'f1'

fig = violin_plot(df, compute_score_between_replicates(df, f1_score), f1_score, 0.).update_layout(width=800)
fig.show()
# save figure as pdf
import plotly.io as pio
pio.write_image(fig, 'f1_score.pdf')

100%|██████████| 1614/1614 [00:02<00:00, 693.64it/s]


In [31]:
def pearson_score(x, y):
    if isinstance(x, np.ndarray) and isinstance(y, np.ndarray):
        A, B = x, y
    else:
        A, B = x['sub_rate'].values[0], y['sub_rate'].values[0]
    mask = (A != -1000.) & (B != -1000.)
    A, B = A[mask], B[mask]
    return np.corrcoef(A, B)[0, 1]

pearson_score.name = 'pearson'


fig = violin_plot(df, compute_score_between_replicates(df, pearson_score), pearson_score, 0.).update_layout(width=800)#.show()
fig.show()
# save figure as pdf
import plotly.io as pio
pio.write_image(fig, 'pearson_score.pdf')

100%|██████████| 1614/1614 [00:00<00:00, 3424.18it/s]


## all-together layout

In [32]:
fig1 = violin_plot(df, compute_score_between_replicates(df, pearson_score), pearson_score, 0.)
fig2 = violin_plot(df, compute_score_between_replicates(df, f1_score), f1_score, 0.).update_layout(width=800)

100%|██████████| 1614/1614 [00:02<00:00, 692.25it/s]


In [38]:
from plotly.subplots import make_subplots
import plotly
colors = plotly.colors.qualitative.Plotly

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.2)

def post_process_trace(fig, trace, row, col):
    # group traces by name
    trace.name = 'mRNA' if 'mRNA' in trace.name else 'pri-miRNA'
    trace.legendgroup = trace.name
    # set cxolor
    trace.marker.color = colors[2] if 'mRNA' in trace.name else colors[1]
    trace.showlegend = row == 0
    # change data color
    fig.add_trace(trace, row=row, col=col)
    # remove y axis text
    fig.update_xaxes(showline=True, linewidth=2, linecolor='lightgrey', mirror=True, showgrid=False, gridcolor='white', gridwidth=2, row=row, col=col, range=[0, 1], title_text=("pearson score" if row == 1 else "f1 score") + " between biological replicates")
    # no text on y axis
    fig.update_yaxes(showline=True, linewidth=2, linecolor='lightgrey', mirror=True, showgrid=False, gridcolor='white', gridwidth=2, row=row, col=col)
    fig.update_layout(
        width=800,  
        height=600,  
        paper_bgcolor='white',  # Background color of the entire plot
        plot_bgcolor='white',  # Background color of the plot area
        # add a frame
        
        # put less distance between violins
        violingap=0,
        
        # remove outliers 
        
        margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins
        font=dict(
        family='helvetica neue',
        size=16,
        
    ),)
    return fig

for trace in fig1.data:
    post_process_trace(fig, trace, 1, 1)
for trace in fig2.data:
    post_process_trace(fig, trace, 2, 1)


# make legend horizontal
# save to pdf
import plotly.io as pio
pio.write_image(fig, 'images/c_data_quality.pdf')
fig.show()

In [ ]:
import os

def make_line(library, name):
    if 'Un' in name:
        plate = name.replace('-', '_').split('_')[1]
    else:
        plate = name.split('_')[1]
    replicate = name.split('_')[2] if len(name.split('_')) > 2 else None
    return dict(
        library=library + '/' + name,
        title = f"{library}, plate {plate}, [DMS] 1.5%, replicate {replicate}",
        organism = 'in vitro',
        molecule = 'Total RNA',
        single_or_paired = 'paired-end',
        processed = f"{library}/{library}.json",
        raw1 = f"{library}/{name}_S5_L001_R1_001.fastq.gz",
        raw2 = f"{library}/{name}_S5_L001_R2_001.fastq.gz",
        fasta = f"{library}/fasta/1-5_{plate}.fasta",
    )
    
df = pd.DataFrame([make_line('mRNA', name) for name in os.listdir('/Users/yvesmartin/Downloads/mRNA') if not '.' in name])
df.to_csv('mRNA.csv', index=False)

## Bootstrapping on F1 score

In [ ]:
df_pri_miRNA = pd.DataFrame.from_dict(rouskinhf.get_dataset('pri_miRNA'), orient='index').reset_index().rename(columns={'index':'reference'})
df_pri_miRNA['dataset'] = 'pri_miRNA'

# load human_mRNA from HF
df_human_mRNA = pd.DataFrame.from_dict(rouskinhf.get_dataset('human_mRNA'), orient='index').reset_index().rename(columns={'index':'reference'})
df_human_mRNA['dataset'] = 'human_mRNA'

df_hf = pd.concat([df_pri_miRNA, df_human_mRNA])

df_hf = pd.merge(df_hf, df[['reference', 'n_reads']], on=['reference'], how='right')

df_hf['reads'] = df_hf['n_reads'].astype(int)
df_hf['dms'] = df_hf['dms'].apply(lambda x: tuple(x))
df_hf = df_hf.groupby(['reference', 'sequence', 'dms', 'dataset']).aggregate({'reads': 'sum'}).reset_index()

# shuffle 
df_hf = df_hf.sample(frac=1, replace=False).reset_index(drop=True)

KeyError: "None of [Index(['reference', 'n_reads'], dtype='object')] are in the [columns]"

In [ ]:
# define a bootstrapping function using a binomial distribution

import numpy as np
import tqdm

n_bootstrap = 1

def bootstrap(n, p, N=n_bootstrap):
    mask = p != -1000.
    n = (np.ones(np.sum(mask)) * n).astype(int)
    p_boot = np.random.binomial(n, p[mask], (N, np.sum(mask))) / n
    out = np.ones((N, len(p))) * -1000.
    out[:, mask] = p_boot
    return out

f1_scores_total = []
r2_scores_total = []
refs = []
datasets = []
for idx, line in tqdm.tqdm(df_hf.iterrows(), total=len(df_hf)):
    reference, sequence, dms, dataset, reads = line[['reference', 'sequence', 'dms', 'dataset', 'reads']]
    dms = np.array(dms) 
    f1_scores_local = []
    r2_scores_local = []
    for dms_boot in bootstrap(reads, dms):
        pred = rna.predictStructure(sequence, dms=dms_boot)
        f1_scores_local.append(f1_score_sklearn(dot2bool(pred), dot2bool(pred)))
        r2_scores_local.append(pearson_score(dms, dms_boot))
        
    f1_scores_total.append(f1_scores_local)
    r2_scores_total.append(r2_scores_local)
    refs.append(reference)
    datasets.append(dataset)
    if idx > 1:
        break
    
scores = pd.DataFrame({
    'reference': [ref for _ in range(n_bootstrap) for ref in refs],
    'f1': [f1 for f1_scores_local in f1_scores_total for f1 in f1_scores_local],``
    'pearson': [r2 for r2_scores_local in r2_scores_total for r2 in r2_scores_local],
    'dataset': [dataset for _ in range(n_bootstrap) for dataset in datasets],
})



In [ ]:
scores = pd.read_feather('data_quality_bootstrapping.feather')
violin_plot(
    df_hf, 
    scores,
    f1_score,
    0.
).update_layout(title = 'F1 score using RNAstructre and bootstrapped DMS signal').show()

violin_plot(
    df_hf, 
    scores,
    pearson_score,
    0.
).update_layout(title = 'Pearson score using RNAstructre and bootstrapped DMS signal').show()

In [ ]:
# import checksum MD5
import hashlib

def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b''):
            hash_md5.update(chunk)
            
    return hash_md5.hexdigest()

# check if the files are the same
assert md5('data_quality.feather') == md5('data_quality.feather')

import os
import pandas as pd
import tqdm

checksums = {}
# walk through /users/yvesmartin/data/dl_paper_data
for root, dirs, files in tqdm.tqdm(os.walk('/Users/yvesmartin/data/dl_paper_data')):
    for name in files:
        if name.endswith('.xlsx') or name.startswith('.'):
            continue
        file_w_path = os.path.join(root, name)
        checksums[file_w_path] = md5(file_w_path)
    


38it [18:08, 28.64s/it] 


In [ ]:
df = pd.DataFrame(checksums.items(), columns=['file', 'checksum'])
# remove /Users/yvesmartin/data/dl_paper_data/ from the path
df['file'] = df['file'].apply(lambda x: '/'.join(x.split('/')[5:]))
df.to_csv('checksums.csv', index=False)